In [3]:
pip install mlxtend

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------  1.4/1.4 MB 46.2 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 30.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [7]:
import sqlite3
import pandas as pd

con = sqlite3.connect("accident_data_v1.0.0_2023.db")
cur = con.cursor()

table_list = [a[0] for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]
sql_query = f"SELECT * FROM {table_list[0]}"
accident_df = pd.read_sql_query(sql_query, con)

In [9]:
columns_name = ['accident_severity', 'weather_conditions', 'speed_limit']
ap_df = accident_df[columns_name]

In [10]:
ap_df= ap_df.fillna('unknown')

In [12]:
severity = pd.get_dummies(accident_df.accident_severity, prefix = "severity")

In [13]:
display(severity)

,severity_1,severity_2,severity_3
0,True,False,False
1,False,False,True
2,False,False,True
3,False,False,True
4,False,True,False
...,...,...,...
461347,False,True,False
461348,False,False,True
461349,False,True,False
461350,False,False,True


In [15]:
weather = pd.get_dummies(accident_df.weather_conditions, prefix = "weather")
display(weather)

,weather_-1,weather_1,weather_2,weather_3,weather_4,weather_5,weather_6,weather_7,weather_8,weather_9
0,False,True,False,False,False,False,False,False,False,False
1,False,True,False,False,False,False,False,False,False,False
2,False,True,False,False,False,False,False,False,False,False
3,False,False,True,False,False,False,False,False,False,False
4,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
461347,False,True,False,False,False,False,False,False,False,False
461348,False,True,False,False,False,False,False,False,False,False
461349,False,True,False,False,False,False,False,False,False,False
461350,False,True,False,False,False,False,False,False,False,False


In [17]:
speed = pd.get_dummies(accident_df.speed_limit, prefix = "speed")
display(speed)

,speed_-1,speed_20,speed_30,speed_40,speed_50,speed_60,speed_70
0,False,False,True,False,False,False,False
1,False,False,True,False,False,False,False
2,False,False,True,False,False,False,False
3,False,False,True,False,False,False,False
4,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...
461347,False,False,True,False,False,False,False
461348,False,False,True,False,False,False,False
461349,False,False,False,False,False,True,False
461350,False,False,True,False,False,False,False


In [18]:
ap_df.head()

,accident_severity,weather_conditions,speed_limit
0,1,1,30
1,3,1,30
2,3,1,30
3,3,2,30
4,2,1,20


In [41]:
frames = [severity, weather, speed]  # Replace with your actual dataframes
apri_df = pd.concat(frames, axis = 1)


In [42]:
apri_df.head()

,severity_1,severity_2,severity_3,weather_-1,weather_1,weather_2,weather_3,weather_4,weather_5,weather_6,weather_7,weather_8,weather_9,speed_-1,speed_20,speed_30,speed_40,speed_50,speed_60,speed_70
0,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
1,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
2,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
3,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False
4,False,True,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False


In [47]:
freq_item_sets = apriori(apri_df, min_support = 0.2, use_colnames = True)
print(freq_item_sets)

    support                           itemsets
0  0.798299                       (severity_3)
1  0.795436                        (weather_1)
2  0.595235                         (speed_30)
3  0.631260            (weather_1, severity_3)
4  0.486100             (speed_30, severity_3)
5  0.476458              (speed_30, weather_1)
6  0.387476  (weather_1, speed_30, severity_3)


In [51]:
rules = association_rules(freq_item_sets, metric = 'lift', min_threshold = 0.5)
display(rules)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(weather_1),(severity_3),0.795436,0.798299,0.631260,0.793602,0.994116,-0.003736,0.977243,-0.028119
1,(severity_3),(weather_1),0.798299,0.795436,0.631260,0.790756,0.994116,-0.003736,0.977633,-0.028507
2,(speed_30),(severity_3),0.595235,0.798299,0.486100,0.816651,1.022989,0.010924,1.100092,0.055518
3,(severity_3),(speed_30),0.798299,0.595235,0.486100,0.608919,1.022989,0.010924,1.034989,0.111412
4,(speed_30),(weather_1),0.595235,0.795436,0.476458,0.800454,1.006308,0.002987,1.025146,0.015487
5,(weather_1),(speed_30),0.795436,0.595235,0.476458,0.598990,1.006308,0.002987,1.009363,0.030644
6,"(speed_30, weather_1)",(severity_3),0.476458,0.798299,0.387476,0.813243,1.018719,0.007120,1.080016,0.035098
7,"(weather_1, severity_3)",(speed_30),0.631260,0.595235,0.387476,0.613814,1.031213,0.011728,1.048109,0.082085
8,"(speed_30, severity_3)",(weather_1),0.486100,0.795436,0.387476,0.797113,1.002109,0.000815,1.008267,0.004094
9,(weather_1),"(speed_30, severity_3)",0.795436,0.486100,0.387476,0.487124,1.002109,0.000815,1.001998,0.010286
